In [115]:
import numpy as np
import pandas as pd

In [116]:
from sklearn import ensemble

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

# Load data

In [117]:
!ls ../training_data/

psl_df_cleaned.csv	       va_df_houston.csv
road_impediments_gbgeo.csv.gz  va_df_la.csv
va_df_atlanta.csv	       va_df_raleigh.csv
va_df_denver.csv	       vehicle_activity_dense_long_lat.csv.zip
va_df_durham.csv	       vehicle_activity_with_distances.csv.gz


In [118]:
cities = ['atlanta', 'denver', 'durham', 'houston', 'la', 'raleigh']

In [119]:
DATA = {}

for city in cities:
    DATA[city] = pd.read_csv(f'../training_data/va_df_{city}.csv')

In [120]:
display(DATA['la'].head(2))
print(DATA['la'].shape)

,Unnamed: 0,geohash,lat,long,city,average_idle_total,median_idle_total,cumulative_daily_idle_total,average_park_time,average_park_time_ratio,total_searching,percent_searching
0,0,9q5cu0z,34.06,-118.29,Los Angeles,7.2526,5.9167,75.0995,6.3136,1.4596,11.0,0.0940
1,1,9q5cf6b,34.07,-118.38,Los Angeles,8.2758,7.1667,119.3317,6.2867,1.7713,5.0,0.0568


(18302, 12)


## `road_impediments_gbgeo`

In [121]:
road_impediments_gbgeo = pd.read_csv('../training_data/road_impediments_gbgeo.csv.gz')

In [122]:
road_impediments_gbgeo.columns

Index(['Unnamed: 0', 'geohash', 'average_acceleration-min',
       'average_acceleration-max', 'average_acceleration-mean',
       'average_acceleration-median', 'percent_vehicles_acceleration-min',
       'percent_vehicles_acceleration-max',
       'percent_vehicles_acceleration-mean',
       'percent_vehicles_acceleration-median', 'average_monthly_vehicles-min',
       'average_monthly_vehicles-max', 'average_monthly_vehicles-mean',
       'average_monthly_vehicles-median', 'percent_car-min', 'percent_car-max',
       'percent_car-mean', 'percent_car-median', 'percent_mpv-min',
       'percent_mpv-max', 'percent_mpv-mean', 'percent_mpv-median',
       'percent_ldt-min', 'percent_ldt-max', 'percent_ldt-mean',
       'percent_ldt-median', 'percent_mdt-min', 'percent_mdt-max',
       'percent_mdt-mean', 'percent_mdt-median', 'percent_hdt-min',
       'percent_hdt-max', 'percent_hdt-mean', 'percent_hdt-median',
       'percent_other-min', 'percent_other-max', 'percent_other-mean',
      

In [123]:
road_impediments_gbgeo_columns = [ 'average_acceleration-min',
       'average_acceleration-max', 'average_acceleration-mean',
       'average_acceleration-median', 'percent_vehicles_acceleration-min',
       'percent_vehicles_acceleration-max',
       'percent_vehicles_acceleration-mean',
       'percent_vehicles_acceleration-median', 'average_monthly_vehicles-min',
       'average_monthly_vehicles-max', 'average_monthly_vehicles-mean',
       'average_monthly_vehicles-median', 'percent_car-min', 'percent_car-max',
       'percent_car-mean', 'percent_car-median', 'percent_mpv-min',
       'percent_mpv-max', 'percent_mpv-mean', 'percent_mpv-median',
       'percent_ldt-min', 'percent_ldt-max', 'percent_ldt-mean',
       'percent_ldt-median', 'percent_mdt-min', 'percent_mdt-max',
       'percent_mdt-mean', 'percent_mdt-median', 'percent_hdt-min',
       'percent_hdt-max', 'percent_hdt-mean', 'percent_hdt-median',
       'percent_other-min', 'percent_other-max', 'percent_other-mean',
       'percent_other-median',
]

## `vehicle_activity_with_distances`

In [124]:
vehicle_activity_with_distances = pd.read_csv('../training_data/vehicle_activity_with_distances.csv.gz')

/home/strokach/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [125]:
display(vehicle_activity_with_distances.head(2))
print(vehicle_activity_with_distances.shape)
print(len(set(vehicle_activity_with_distances['geohash'])))

,Unnamed: 0,geohash,city,county,state,metro,average_idle_total,median_idle_total,cumulative_daily_idle_total,average_idle_car,...,hospital_k5,sports_venue_k1,sports_venue_k3,sports_venue_k5,church_k1,church_k3,church_k5,school_k1,school_k3,school_k5
0,21,9q5cu0z,Los Angeles,Los Angeles,California,"Los Angeles-Long Beach-Anaheim, CA",7.2526,5.9167,75.0995,8.1452,...,1.858249,2.820807,4.040846,6.344712,0.815988,2.311438,4.060180,0.346995,0.409768,0.446790
1,57,9q5cf6b,Los Angeles,Los Angeles,California,"Los Angeles-Long Beach-Anaheim, CA",8.2758,7.1667,119.3317,5.5510,...,2.368051,6.152981,9.133728,10.011422,2.129068,3.572366,5.162235,1.296467,1.527772,1.772008


(78439, 75)
78439


In [126]:
vehicle_activity_with_distances.columns

Index(['Unnamed: 0', 'geohash', 'city', 'county', 'state', 'metro',
       'average_idle_total', 'median_idle_total',
       'cumulative_daily_idle_total', 'average_idle_car', 'average_idle_mpv',
       'average_idle_ldt', 'average_idle_mdt', 'average_idle_hdt',
       'hourly_distribution_idle', 'percent_car_idle', 'percent_mpv_idle',
       'percent_ldt_idle', 'percent_mdt_idle', 'percent_hdt_idle',
       'percent_other_idle', 'percent_gas_idle', 'percent_diesel_idle',
       'percent_other_engine_idle', 'average_latitude_park',
       'average_longitude_park', 'average_park_time',
       'average_park_time_ratio', 'total_searching', 'percent_searching',
       'average_unique_geohashes', 'average_total_geohashes',
       'circling_distribution', 'hourly_distribution_park', 'percent_car_park',
       'percent_mpv_park', 'percent_ldt_park', 'percent_mdt_park',
       'percent_hdt_park', 'percent_other_park', 'latitude', 'longitude',
       'closest_fuel_station_k1', 'closest_fuel_sta

In [127]:
vehicle_activity_with_distances_columns = [
    'closest_fuel_station_k1', 'closest_fuel_station_k3',
       'closest_fuel_station_k5', 'cell_coverage_dark_spot_k1',
       'cell_coverage_dark_spot_k3', 'cell_coverage_dark_spot_k5',
       'heavy_duty_parking_k1', 'heavy_duty_parking_k3',
       'heavy_duty_parking_k5', 'service_center_metrics_k1',
       'service_center_metrics_k3', 'service_center_metrics_k5',
       'public_services_locations_k1', 'public_services_locations_k3',
       'public_services_locations_k5', 'cell_structure_k1',
       'cell_structure_k3', 'cell_structure_k5', 'college_k1', 'college_k3',
       'college_k5', 'hospital_k1', 'hospital_k3', 'hospital_k5',
       'sports_venue_k1', 'sports_venue_k3', 'sports_venue_k5', 'church_k1',
       'church_k3', 'church_k5', 'school_k1', 'school_k3', 'school_k5',
    
]

# Train model for city

In [163]:
data = (
    DATA['la']
    .merge(road_impediments_gbgeo[['geohash'] + road_impediments_gbgeo_columns], on=['geohash'], how='left')
    .merge(vehicle_activity_with_distances[['geohash'] + vehicle_activity_with_distances_columns], on=['geohash'])
)

In [164]:
assert len(data) == len(DATA['la'])

In [165]:
feature_names = road_impediments_gbgeo_columns + vehicle_activity_with_distances_columns

In [167]:
null_mask = data[feature_names + ['average_idle_total']].isnull().any(axis=1)

In [168]:
null_mask.sum()

1106

In [169]:
data = data[~null_mask]

In [170]:
X = data[road_impediments_gbgeo_columns + road_impediments_gbgeo_columns]

In [171]:
Y = data['average_idle_total']

In [172]:
clf = ensemble.GradientBoostingRegressor(
    learning_rate=0.1, n_estimators=3_000, subsample=1.0)

In [173]:
clf.fit(X, Y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [174]:
clf_fit = _

In [175]:
clf_fit

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=3000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [176]:
plt.bar(clf.feature_importances_

SyntaxError: unexpected EOF while parsing (<ipython-input-176-417e13b77977>, line 1)

In [ ]:
fg, axs = plt.subplots(figsize=(12, 16))
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(feature_names)[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')

In [183]:
import pickle

In [181]:
os.makedirs('machine_learning')

In [182]:
import os

In [184]:
with open('machine_learning/clf_1.pickle', 'wb') as fout:
    pickle.dump(clf, fout)